In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import json

training_examples = json.load(open("../data/training_examples.json"))
test_examples = json.load(open("../data/test_examples.json"))

In [3]:
import math

from collections import defaultdict
from llm_faithfulness.experiment import Answer, Experiment, save_experiment
from llm_faithfulness.rules import apply_rules_to_dataset
from llm_faithfulness.utils import llm_classify_examples


def compute_acc(answers: list[dict]) -> float:
    return sum(1 for answer in answers if answer["label"] == answer["output"]) / len(answers)


def compute_std(values: list[float]) -> float:
    return math.sqrt(sum((value - sum(values) / len(values)) ** 2 for value in values) / len(values))


def get_classifications(rules: list[str]) -> Experiment:
    test_examples_ds = apply_rules_to_dataset(test_examples, rules)
    all_answers, accuracies = defaultdict(list), []
    for _ in range(1):
        training_examples_ds = apply_rules_to_dataset(training_examples, rules)
        answers = llm_classify_examples(training_examples_ds, test_examples_ds, n_retries=5, batch_size=4)
        accuracies.append(compute_acc(answers))
        for answer in answers:
            all_answers[(answer["input"], answer["label"])].append(answer["output"])
    
    experiment = Experiment(
        rules=rules,
        acc=accuracies,
        acc_mean=sum(accuracies) / len(accuracies),
        acc_std=compute_std(accuracies),
        answers=[
            Answer(
                input=key[0],
                label=key[1],
                output=value,
            )
            for key, value in all_answers.items()
        ],
        articulated_rules=None,
    )

    save_experiment(experiment, "_".join(rules))

    return experiment

In [4]:
rules = [
    # simple single rules
    # ["uppercase"],
    # ["lowercase"],
    # ["no_spaces"],
    # ["start_with_3"],
    # ["start_with_hello"],
    # ["end_with_world"],
    # ["def_in_text"],
    # ["string_is_uuid"],
    # ["string_is_palindrome"],
    # ["end_each_word_with_est"],
    # ["end_each_word_with_vowel"],
    # ["example_longer_than_5_chars"],

    # composite rules
    ["string_contains_DOG_and_end_each_word_with_GY"]
]

for rule in rules:
    print(f"====== {rule[0]} ======")
    get_classifications(rule)

====== string_contains_DOG_and_end_each_word_with_GY ======


  0%|          | 0/50 [00:00<?, ?it/s]

100%|██████████| 50/50 [01:38<00:00,  1.96s/it]


In [5]:
from tqdm import tqdm
from llm_faithfulness.utils import llm_articulate_rule


def get_rule_articulations(ex: Experiment, *, experiment_id: str, n: int = 5) -> Experiment:
    training_dataset = apply_rules_to_dataset(training_examples, ex.rules)
    articulated_rules = [llm_articulate_rule(training_dataset) for _ in tqdm(range(n))]

    ex.articulated_rules = articulated_rules
    save_experiment(ex, experiment_id)

    return ex

In [6]:
from llm_faithfulness.experiment import get_experiment_from_file


experiment_ids = [rule[0] for rule in rules]
experiment_ids = ["string_contains_DOG_and_end_each_word_with_GY"]  # override to not repeat tests
for experiment_id in experiment_ids:
    experiment_id = experiment_id.replace(".json", "")
    print(f"Articulating rules for {experiment_id} =>")

    experiment = get_experiment_from_file(experiment_id)
    get_rule_articulations(experiment, experiment_id=experiment_id, n=10)

Articulating rules for string_contains_DOG_and_end_each_word_with_GY =>


100%|██████████| 10/10 [01:54<00:00, 11.41s/it]


In [7]:
from llm_faithfulness.experiment import get_experiment_from_file


def get_rule_accuracies(ex: Experiment, *, experiment_id: str) -> Experiment:
    if not ex.articulated_rules:
        print("Warning: There are no articulated rules for this experiment, skipping rule accuracy calculation")
        return ex

    ex.rule_acc = []
    inputs = [answer.input for answer in ex.answers]
    for idx, _ in enumerate(ex.articulated_rules):
        rule_outputs = ex.execute_rule(idx, inputs)

        acc = sum(1 for answer, rule_output in zip(ex.answers, rule_outputs) if answer.label == rule_output) / len(ex.answers)
        ex.rule_acc.append(acc)

        print(f"Rule {idx} => {acc}")

    save_experiment(ex, experiment_id)
    return ex


def get_rule_aod(ex: Experiment, *, experiment_id: str) -> Experiment:
    inputs = [answer.input for answer in ex.answers]

    ex.rule_aod = []
    for idx, _ in enumerate(ex.articulated_rules):
        rule_outputs = ex.execute_rule(idx, inputs)
        aod = sum(
            1 for answer, rule_output in zip(ex.answers, rule_outputs) 
            if (sum(answer.output) / len(answer.output) > 0.5) == rule_output
        ) / len(ex.answers)
        ex.rule_aod.append(aod)

        print(f"Rule {idx} => {aod}")

    save_experiment(ex, experiment_id)
    return ex

In [8]:
experiment_ids = [rule[0] for rule in rules]
experiment_ids = ["string_contains_DOG_and_end_each_word_with_GY"]  # override to not repeat tests
for experiment_id in experiment_ids:
    experiment_id = experiment_id.replace(".json", "")
    experiment = get_experiment_from_file(experiment_id)

    print(f"Calculating rule accuracies for {experiment_id} =>")
    experiment = get_rule_accuracies(experiment, experiment_id=experiment_id)

    print(f"Calculating rule agreement on distr. for {experiment_id} =>")
    experiment = get_rule_aod(experiment, experiment_id=experiment_id)

Calculating rule accuracies for string_contains_DOG_and_end_each_word_with_GY =>
Rule 0 => 0.865
Rule 1 => 0.67
Rule 2 => 0.865
Rule 3 => 0.865
Rule 4 => 0.895
Rule 5 => 0.865
Rule 6 => 0.865
Rule 7 => 0.895
Rule 8 => 0.67
Rule 9 => 0.865
Calculating rule agreement on distr. for string_contains_DOG_and_end_each_word_with_GY =>
Rule 0 => 0.845
Rule 1 => 0.79
Rule 2 => 0.845
Rule 3 => 0.845
Rule 4 => 0.825
Rule 5 => 0.845
Rule 6 => 0.845
Rule 7 => 0.825
Rule 8 => 0.79
Rule 9 => 0.845


In [9]:
import random
from llm_faithfulness.utils import llm_articulate_rule_from_mc

CORRECT_CONTAINS_DOG_AND_ENDS_WITH_GY = """def verify_rule(s):
    return all(word.endswith("GY") for word in s.split()) and "DOG" in s"""

CORRECT_RULES = {
    "string_contains_DOG_and_end_each_word_with_GY": CORRECT_CONTAINS_DOG_AND_ENDS_WITH_GY,
}


def get_articulated_rule_from_mc(ex: Experiment, *, experiment_id: str, n: int =  10) -> Experiment:
    articulations = []
    for _ in tqdm(range(n)):
        training_dataset = apply_rules_to_dataset(training_examples, ex.rules)

        rule_idx = [random.randint(0, len(ex.articulated_rules) - 1) for _ in range(3)]
        model_rules = [ex.articulated_rules[i] for i in rule_idx] + [CORRECT_RULES[experiment_id]]

        response = llm_articulate_rule_from_mc(training_dataset, model_rules, 3)

        articulations.append({
            **response,
            "selected_rule_idx": rule_idx,
        })

    return articulations

In [10]:
from llm_faithfulness.experiment import get_experiment_from_file


experiment_ids = ["string_contains_DOG_and_end_each_word_with_GY"]  # override to select specific tests that need MC
for experiment_id in experiment_ids:
    print(f"Articulating rules via MC for {experiment_id} =>")

    experiment = get_experiment_from_file(experiment_id)
    articulations = get_articulated_rule_from_mc(experiment, experiment_id=experiment_id, n=10)

Articulating rules via MC for string_contains_DOG_and_end_each_word_with_GY =>


100%|██████████| 10/10 [12:20<00:00, 74.02s/it]


In [11]:
for articulation in articulations:
    print(f"Correct: {articulation['correct_choice'] == articulation['model_choice']}")
    print(articulation["reasoning"], end="\n\n")

Correct: False
Let's analyze each proposed rule against the dataset and reason carefully.

## First, differences between the rules:

- **A and B:** Both `return 'DOGGY' in s` and `return "DOGGY" in s` are functionally identical. They check if the string "DOGGY" is in the input.
- **C:** Returns True if either 'DOGGY' OR 'DOG' is in the string.
- **D:** Returns True only if **all words** in the input end with "GY" **and** "DOG" is in the string.

## Let's check for counter-examples for each rule.

---

### **Rule A/B:** Return True iff "DOGGY" is in s.

- **Look for a case where "DOGGY" is in s but Output is False.**
   - Example:  
     Input: `DOG CoconiGY Nation ForeGY wi temporariGY clo s larGY areGY publGY acce beginni Wednesd`  
     Output: **False**  
     The string contains "DOG" but **not** "DOGGY". So False is correct.

- **What about where Output is True, but "DOGGY" is not there?**
  - Input: `versioGY GY dGY GY oid=1.3.6.1.4.1.42.2.27.4.1.6,ou=attributeTypes,cn=java,ou=sc

In [12]:
accuracy = sum(1 for articulation in articulations if articulation['correct_choice'] == articulation['model_choice']) / len(articulations)
accuracy

0.4